<a href="https://colab.research.google.com/github/AbhishekRP2002/llm-learning-cookbook/blob/main/langchain/RAG_from_Scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install langchain_community tiktoken langchain-openai langchainhub chromadb langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 34.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 509.0/509.0 kB 41.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.9/815.9 kB 54.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.2/241.2 kB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 kB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 28.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.1/41.1 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 44.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install qdrant-client

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 206.3/206.3 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 32.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 7.7 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-metadata 1.14.0 requires protobuf<4.21,>=3.20.3, but you have protobuf 4.25.3 which is incompatible.


In [ ]:
import os
from getpass import getpass
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.langchain.plus'
os.environ['OPENAI_API_KEY'] = getpass("OpenAI API Key:")
os.environ['LANGCHAIN_API_KEY'] = getpass("Langchain API Key :")

OpenAI API Key:··········
Langchain API Key :··········


In [ ]:
# langchain_api_key = getpass("Enter your Langchain API Key :")

In [ ]:

import bs4
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader,PySparkDataFrameLoader,ArxivLoader,TextLoader
from langchain_community.vectorstores import Chroma,Qdrant
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

In [ ]:
# First load your documents on using which you wanna fetch your query response
loader = WebBaseLoader(
    web_paths = ["https://medium.com/international-school-of-ai-data-science/implementing-rag-with-langchain-and-hugging-face-28e3ea66c5f7",
                 "https://medium.com/@akriti.upadhyay/integrating-llamaindex-and-qdrant-similarity-search-for-patient-record-retrieval-7090e77b971e",
                 "https://medium.com/international-school-of-ai-data-science/navigating-gemini-a-family-of-multimodal-intelligence-b80d1419e4d7",
                 "https://medium.com/international-school-of-ai-data-science/optimizing-performance-with-peft-a-deep-dive-into-prompt-tuning-2b9a17bc9851",
                 "https://medium.com/@akriti.upadhyay/how-to-create-a-vector-based-movie-recommendation-system-using-qdrant-db-e026554f49ec"
                 ]

)
documents = loader.load()

In [ ]:
# now split the documents into chunks and then create their embeddings
text_splitter =  RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splitted_docs = text_splitter.split_documents(documents)
embeddings = OpenAIEmbeddings()
qdrant_vector_store = Qdrant.from_documents(
    splitted_docs,
    embeddings,
    location=":memory:",  # Local mode with in-memory storage only
    collection_name="my_documents"

)
chroma_vectore_store = Chroma.from_documents(documents=splitted_docs,
                                    embedding=embeddings)

In [ ]:
qdrant_retriever = qdrant_vector_store.as_retriever() # VectorStore Retriver -- I can use other Retrievers as well
chroma_retriever = chroma_vectore_store.as_retriever()
# TODO : Use MultiVector Retriver , Self Query and ParentDocument

In [ ]:
prompt = hub.pull("rlm/rag-prompt") # TODO : Add your own prompt in Langchain hub and use that


In [ ]:
llm = ChatOpenAI()

rag_chain = (
    {"context": qdrant_retriever , "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
rag_chain.invoke("What can I learn from the above Medium articles about Large Language Models ?Consider all the articles for generating your response ")

'From the above Medium articles, you can learn about optimizing performance with prompt tuning in Large Language Models (LLM) by adjusting parameters and incorporating human feedback. You can also learn about the challenges of using open-source models and the benefits of integrating external data to improve the accuracy and relevance of LLM responses. Additionally, the articles discuss the implementation of Retrieval Augmented Generation (RAG) to enhance the effectiveness of LLM applications by providing context and reducing inaccuracies.'